# Importações

In [1]:
import pandas as pd
from os import listdir
from os.path import isfile, join
import numpy as np
from sklearn.impute import KNNImputer

# Criando variáveis organizacionais

In [2]:
path = '../raw_data/SP' #caminho geral
files = [f for f in listdir(path) if isfile(join(path, f))] #lista de nomes de arquivos de dados
print(files[0])

INMET_SE_SP_A768_TUPA_01-01-2021_A_31-12-2021.CSV


# Tratamentos iniciais dos dados

## Tratamentos básicos

In [3]:
#---------------------fazer list com os dataframes---------------------
df_list = []
for file in range(0,88):
    df = pd.read_csv(f'../raw_data/SP/{files[file]}', sep=';', skiprows=8, encoding="ISO-8859-1", decimal=',')
    lat_log_alt = pd.read_csv(f'../raw_data/SP/{files[file]}', sep=';', skiprows=4,
                          nrows=3, encoding="ISO-8859-1", decimal=',', names=['lat_lon_alt','valor'])
    df['Estaçao']=files[file].split('_')[4]
    df['Latitude']=lat_log_alt['valor'][0]
    df['Longitude']=lat_log_alt['valor'][1]
    df['Altitude']=lat_log_alt['valor'][2]
    df_list.append(df)
df_list

#---------------------fundir os dataframes no dataframe vazio---------------------
full_df = pd.concat(df_list)
full_df
df2 = full_df.copy()

#---------------------dropar coluna inútil---------------------
df2.drop(columns=["Unnamed: 19"],inplace=True)

#---------------------mudar nome das colunas---------------------
df2= df2.rename(columns={'Data': 'Data',
                                            'Hora UTC': 'Hora(UTC)',
                                            'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)': 'Chuva',
                                            'PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)': 'Pres',
                                            'PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)': 'Pres.max',
                                            'PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)': 'Pres.min',
                                            'RADIACAO GLOBAL (Kj/m²)': 'Radiacao',
                                            'TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)': 'Temp',
                                            'TEMPERATURA DO PONTO DE ORVALHO (°C)': 'Temp.orvalho',
                                            'TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)': 'Temp.max',
                                            'TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)': 'Temp.min',
                                            'TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)': 'Temp.orvalho.max',
                                            'TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)': 'Temp.orvalho.min',
                                            'UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)': 'Umid.max',
                                            'UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)': 'Umid.min',
                                            'UMIDADE RELATIVA DO AR, HORARIA (%)': 'Umid',
                                            'VENTO, DIREÇÃO HORARIA (gr) (° (gr))': 'Dir.vento',
                                            'VENTO, RAJADA MAXIMA (m/s)': 'Rajada.vento',
                                            'VENTO, VELOCIDADE HORARIA (m/s)': 'Vel.vento'})

## Transformação de dados e preenchimento

### Formatando data e hora

In [4]:
#---------------------colocando data e hora no formato adequado---------------------
#df2["Data"] = pd.to_datetime(df2["Data"])
df2['datahora'] = df2.apply(lambda row: pd.to_datetime(f'{row["Data"]} {row["Hora(UTC)"]}'), axis=1)

### Transformando chuva em variável categórica

In [5]:
#---------------------transformando a chuva em variável categórica---------------------
def classe_chuva(precipitacao):
    mm=precipitacao
    if mm == "NaN":
        chuva = "NaN"
    if mm == 0:
        chuva = 'nao chove'
    elif mm >0 and mm <=2.5:
        chuva = 'fraca'
    elif mm >2.5 and mm<=7.5:
        chuva = 'moderada'
    elif mm >7.5 and mm<=50:
        chuva = 'forte'
    else:
        chuva = 'muito forte'
    return chuva
df2['classe_chuva'] = df2['Chuva'].apply(lambda x: classe_chuva(x))

### Tratando valores nulos de radiação

In [6]:
#---------------------Tratando valores nulos de radiação---------------------
def tratar_radiacao(hora,radiacao):
    if (hora.hour >= 22) and (hora.hour <= 8):
        if str(radiacao) == "NaN":
            radiacao = 0
    return radiacao
df2['Radiacao'] = df2.apply(lambda x: tratar_radiacao(x['datahora'],x['Radiacao']),axis=1)

### Imputer da variável de vento

In [7]:
#---------------------fazer imputer nas colunas de vento---------------------
imputer = KNNImputer(n_neighbors=2)
imputer.fit_transform(df2[['Rajada.vento', 'Vel.vento', 'Dir.vento']])

KeyboardInterrupt: 

# Explorando os dados

In [ ]:
df2

In [ ]:
df2['datahora'].dt.hour


In [ ]:
df2

In [ ]:
#verificando células nulas
df2.isnull().sum().sort_values(ascending=False)/len(df2)